In [ ]:
import rosbag
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
# Load rosbag
bag_path = '/workspace/rosbags/2025-12-17-16-02-22.bag'
bag = rosbag.Bag(bag_path)

# Get bag info
print(f"Bag: {bag_path}")
print(f"Duration: {bag.get_end_time() - bag.get_start_time():.2f} seconds")
print(f"\nTopics:")
for topic, topic_info in bag.get_type_and_topic_info().topics.items():
    print(f"  {topic}: {topic_info.message_count} messages ({topic_info.msg_type})")

In [ ]:
# Extract timestamps from each sensor
timestamps = defaultdict(list)

# Adjust these topic names based on your actual rosbag
topics_to_analyze = [
    '/ti_mmwave/radar_scan',  # Radar
    # '/mocap/pose',           # MoCap (adjust name)
    # '/imu/data',             # IMU (adjust name)
]

for topic, msg, t in bag.read_messages(topics=topics_to_analyze):
    # Store both ROS time (t) and message header time if available
    if hasattr(msg, 'header'):
        timestamps[topic].append({
            'ros_time': t.to_sec(),
            'msg_time': msg.header.stamp.to_sec()
        })
    else:
        timestamps[topic].append({
            'ros_time': t.to_sec(),
            'msg_time': t.to_sec()
        })

bag.close()

# Convert to DataFrames
dfs = {}
for topic, data in timestamps.items():
    df = pd.DataFrame(data)
    df['delay'] = df['ros_time'] - df['msg_time']
    dfs[topic] = df
    print(f"\n{topic}:")
    print(f"  Messages: {len(df)}")
    print(f"  Avg delay: {df['delay'].mean()*1000:.2f} ms")
    print(f"  Std delay: {df['delay'].std()*1000:.2f} ms")

In [ ]:
# Plot timestamps and delays
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Message frequency over time
ax = axes[0]
for topic, df in dfs.items():
    ax.plot(df['ros_time'] - df['ros_time'].min(), 
            np.arange(len(df)), 
            label=topic.split('/')[-1], 
            marker='o', markersize=2, alpha=0.6)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Message Count')
ax.set_title('Message Arrival Times')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Time delays
ax = axes[1]
for topic, df in dfs.items():
    ax.plot(df['ros_time'] - df['ros_time'].min(), 
            df['delay'] * 1000,  # Convert to ms
            label=topic.split('/')[-1], 
            marker='o', markersize=2, alpha=0.6)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Delay (ms)')
ax.set_title('Sensor Time Delays')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze radar data in detail
bag = rosbag.Bag(bag_path)
radar_data = []

for topic, msg, t in bag.read_messages(topics=['/ti_mmwave/radar_scan']):
    radar_data.append({
        'time': t.to_sec(),
        'num_points': len(msg.point_id) if hasattr(msg, 'point_id') else 0,
        # Add more fields based on your radar message structure
    })

bag.close()

radar_df = pd.DataFrame(radar_data)
print(f"Radar statistics:")
print(radar_df.describe())

## Next Steps

1. Adjust topic names to match your rosbag
2. Extract relevant data from each sensor type
3. Compute cross-correlation for time delay estimation
4. Visualize motion trajectories